In [1]:
import pandas as pd
pd.options.display.max_colwidth = 30
pd.options.display.max_rows = 6

In [2]:
# Load the CVM table with the registrations of open capital companies
CVM_PATH = "../data/external/cad_cia_aberta.csv"
df = pd.read_csv(
    CVM_PATH, sep=";",
    encoding="iso-8859-1",
    parse_dates=['DT_REG', 'DT_INI_SIT', 'DT_CANCEL']
)
cols = ['DENOM_SOCIAL', 'DT_REG', 'DT_CANCEL', 'SIT', 'DT_INI_SIT', 'CD_CVM',
       'SETOR_ATIV', 'CONTROLE_ACIONARIO']
df = (df
    .loc[:, cols]
    .sort_values('DT_INI_SIT', ignore_index=True)
    .drop_duplicates(subset="CD_CVM", keep="last")
    .sort_values("CD_CVM", ignore_index=True)
)
print('Number of companies available for backtesting', df.CD_CVM.nunique())
df

Number of companies available for backtesting 2448


,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
0,A J RENNER SA IND E PARTIP,1969-06-24,1998-06-17,CANCELADA,1998-06-17,35,Emp. Adm. Participações,PRIVADO
1,ACOPALMA CIA INDL ACOS V P...,1978-05-09,2007-08-07,CANCELADA,2007-08-07,60,Metalurgia e Siderurgia,PRIVADO
2,ACOS ANHANGUERA SA,1971-10-20,1994-02-09,CANCELADA,1994-02-09,78,Metalurgia e Siderurgia,PRIVADO
...,...,...,...,...,...,...,...,...
2445,ANEMUS WIND HOLDING S.A.,2022-06-08,NaT,ATIVO,2022-06-08,26832,Energia Elétrica,PRIVADO
2446,SAP SECURITIZADORA DE CRED...,2022-06-21,NaT,ATIVO,2022-06-21,26840,Securitização de Recebíveis,PRIVADO
2447,CONCESSIONARIA CATARINENSE...,2022-06-21,NaT,ATIVO,2022-06-21,26859,Emp. Adm. Part. - Serviços...,PRIVADO


In [3]:
# Check a company
df.query('DENOM_SOCIAL.str.startswith("VALE")')

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
352,VALE S.A.,1970-01-02,NaT,ATIVO,1970-01-02,4170,Extração Mineral,PRIVADO
1165,VALE FERTILIZANTES S.A.,1992-08-11,2011-12-23,CANCELADA,2011-12-23,14028,Petroquímicos e Borracha,PRIVADO
1384,VALETRON SA,1997-04-23,2015-12-23,CANCELADA,2015-12-23,16225,Emp. Adm. Part. - Sem Seto...,PRIVADO HOLDING


In [4]:
df.CONTROLE_ACIONARIO.unique()

array(['PRIVADO', 'ESTRANGEIRO', 'ESTATAL', 'PRIVADO HOLDING',
       'ESTATAL HOLDING', 'ESTRANGEIRO HOLDING'], dtype=object)

In [5]:
# Remove state owned and foreign companies from selection
company_control = ['PRIVADO', 'PRIVADO HOLDING']
df.query('CONTROLE_ACIONARIO == @company_control', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
0,A J RENNER SA IND E PARTIP,1969-06-24,1998-06-17,CANCELADA,1998-06-17,35,Emp. Adm. Participações,PRIVADO
1,ACOPALMA CIA INDL ACOS V P...,1978-05-09,2007-08-07,CANCELADA,2007-08-07,60,Metalurgia e Siderurgia,PRIVADO
2,ACOS ANHANGUERA SA,1971-10-20,1994-02-09,CANCELADA,1994-02-09,78,Metalurgia e Siderurgia,PRIVADO
...,...,...,...,...,...,...,...,...
2207,ANEMUS WIND HOLDING S.A.,2022-06-08,NaT,ATIVO,2022-06-08,26832,Energia Elétrica,PRIVADO
2208,SAP SECURITIZADORA DE CRED...,2022-06-21,NaT,ATIVO,2022-06-21,26840,Securitização de Recebíveis,PRIVADO
2209,CONCESSIONARIA CATARINENSE...,2022-06-21,NaT,ATIVO,2022-06-21,26859,Emp. Adm. Part. - Serviços...,PRIVADO


In [6]:
# Remove duplicated companies -> adopt last company status as valid
# Remove companies without a sector
df = (df
    .sort_values("DT_INI_SIT", ignore_index=True)
    .dropna(subset='SETOR_ATIV')
    .drop_duplicates(subset="CD_CVM", keep="last", ignore_index=True)
    .sort_values("CD_CVM", ignore_index=True)
)
df

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
0,A J RENNER SA IND E PARTIP,1969-06-24,1998-06-17,CANCELADA,1998-06-17,35,Emp. Adm. Participações,PRIVADO
1,ACOPALMA CIA INDL ACOS V P...,1978-05-09,2007-08-07,CANCELADA,2007-08-07,60,Metalurgia e Siderurgia,PRIVADO
2,ACOS ANHANGUERA SA,1971-10-20,1994-02-09,CANCELADA,1994-02-09,78,Metalurgia e Siderurgia,PRIVADO
...,...,...,...,...,...,...,...,...
2201,ANEMUS WIND HOLDING S.A.,2022-06-08,NaT,ATIVO,2022-06-08,26832,Energia Elétrica,PRIVADO
2202,SAP SECURITIZADORA DE CRED...,2022-06-21,NaT,ATIVO,2022-06-21,26840,Securitização de Recebíveis,PRIVADO
2203,CONCESSIONARIA CATARINENSE...,2022-06-21,NaT,ATIVO,2022-06-21,26859,Emp. Adm. Part. - Serviços...,PRIVADO


In [7]:
df.SETOR_ATIV.value_counts()

Construção Civil, Mat. Constr. e Decoração    165
Máquinas, Equipamentos, Veículos e Peças      157
Emp. Adm. Participações                       136
                                             ... 
Factoring                                       1
Emp. Adm. Part. - Papel e Celulose              1
Emp. Adm. Part. - Bancos                        1
Name: SETOR_ATIV, Length: 68, dtype: int64

In [8]:
for sector in df.SETOR_ATIV.sort_values().unique():
    print(sector)

Agricultura (Açúcar, Álcool e Cana)
Alimentos
Arrendamento Mercantil
Bancos
Bebidas e Fumo
Bolsas de Valores/Mercadorias e Futuros
Brinquedos e Lazer
Comunicação e Informática
Comércio (Atacado e Varejo)
Comércio Exterior
Construção Civil, Mat. Constr. e Decoração
Crédito Imobiliário
Educação
Embalagens
Emp. Adm. Part. - Agricultura (Açúcar, Álcool e Cana)
Emp. Adm. Part. - Alimentos
Emp. Adm. Part. - Arrendamento Mercantil
Emp. Adm. Part. - Bancos
Emp. Adm. Part. - Brinquedos e Lazer
Emp. Adm. Part. - Comunicação e Informática
Emp. Adm. Part. - Comércio (Atacado e Varejo)
Emp. Adm. Part. - Const. Civil, Mat. Const. e Decoração
Emp. Adm. Part. - Crédito Imobiliário
Emp. Adm. Part. - Educação
Emp. Adm. Part. - Embalagens
Emp. Adm. Part. - Energia Elétrica
Emp. Adm. Part. - Extração Mineral
Emp. Adm. Part. - Farmacêutico e Higiene
Emp. Adm. Part. - Gráficas e Editoras
Emp. Adm. Part. - Hospedagem e Turismo
Emp. Adm. Part. - Intermediação Financeira
Emp. Adm. Part. - Metalurgia e Siderurg

In [9]:
removed_sectors = [
    'Arrendamento Mercantil', 
    'Bancos', 
    'Bolsas de Valores/Mercadorias e Futuros', 
    'Crédito Imobiliário', 
    'Emp. Adm. Part. - Arrendamento Mercantil', 
    'Emp. Adm. Part. - Bancos', 
    'Emp. Adm. Part. - Crédito Imobiliário', 
    'Emp. Adm. Part. - Energia Elétrica', 
    'Emp. Adm. Part. - Intermediação Financeira', 
    'Emp. Adm. Part. - Saneamento, Serv. Água e Gás', 
    'Emp. Adm. Part. - Securitização de Recebíveis', 
    'Emp. Adm. Part. - Seguradoras e Corretoras', 
    'Emp. Adm. Part. - Sem Setor Principal', 
    'Emp. Adm. Part.-Bolsas de Valores/Mercadorias e Futuros', 
    'Emp. Adm. Participações', 
    'Energia Elétrica', 
    'Factoring', 
    'Intermediação Financeira', 
    'Saneamento, Serv. Água e Gás', 
    'Securitização de Recebíveis', 
    'Seguradoras e Corretoras', 
    'Telecomunicações'
]

In [10]:
df_excluded = (df
    .query('SETOR_ATIV == @removed_sectors')
    .reset_index(drop=True)
)
df_excluded

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
0,A J RENNER SA IND E PARTIP,1969-06-24,1998-06-17,CANCELADA,1998-06-17,35,Emp. Adm. Participações,PRIVADO
1,ALPART ALPA PARTICIPACOES SA,1982-09-03,1989-10-19,CANCELADA,1989-10-19,310,Emp. Adm. Participações,PRIVADO
2,AMERICA DO SUL LEAS SA ARR...,1982-09-28,2000-03-02,CANCELADA,2000-03-02,388,Arrendamento Mercantil,PRIVADO
...,...,...,...,...,...,...,...,...
758,TRAVESSIA SECURITIZADORA D...,2022-06-08,NaT,ATIVO,2022-06-08,26824,Securitização de Recebíveis,PRIVADO
759,ANEMUS WIND HOLDING S.A.,2022-06-08,NaT,ATIVO,2022-06-08,26832,Energia Elétrica,PRIVADO
760,SAP SECURITIZADORA DE CRED...,2022-06-21,NaT,ATIVO,2022-06-21,26840,Securitização de Recebíveis,PRIVADO


In [11]:
# Check excluded companies
for companie in df_excluded.DENOM_SOCIAL:
    print(companie)

A J RENNER SA IND E PARTIP
ALPART ALPA PARTICIPACOES SA
AMERICA DO SUL LEAS SA ARR MER
AUXILIAR LEASING SA ARR MERC
HSBC SEGUROS BRASIL SA
HSBC LEASING A.M.(BRASIL) SA
BAMERINDUS SA PARTICIPAÇÕES E EMPREENDIMENTOS - EM LIQUIDAÇÃO EXTRAJUDICIAL
BAMERINDUS SA CRED IMOBILIARIO
BAMERINDUS SA FINANC CRED INV
BANCO AGRIMISA SA
BANCO COMERCIAL E DE INVESTIMENTO SUDAMERIS S.A.
BANCO CREFISUL S.A.
BCO AUXILIAR SA
BCO BAMERINDUS INVS SA
BANCO BAMERINDUS BRASIL SA
BANCO BANDEIRANTES DE INVESTIMENTOS SA
UNICARD BANCO MULTIPLO S.A.
BANCO BOAVISTA INTERATLÂNTICO S.A.
BCO BRADESCO INVESTIMENTO SA
BBC ADM E PARTICIPAÇÃO-EM LIQUIDAÇÃO ORDINÁRIA
BANCO BRADESCO S.A.
BANCO ITABANCO S/A
BANCO BOCOM BBM S.A.
BANCO BCN S.A.
BCR BCO CREDITO REAL SA
BCO CREDIPLAN SA
BCO INV AMERICA SUL SA
BANCO ÚNICO S.A.
BANCO COMERCIAL BANCESA S.A.
COMIND PARTICIPACOES SA
BANCO BANEB SA
BANCO BEG S.A.
BANCO DE PERNAMBUCO S.A. - BANDEPE
BANCO DO ESTADO DE SAO PAULO S.A. - BANESPA
BANCO BEA SA
BANCO BRADESCO BBI S.A.
BANCO BAN

In [12]:
df_included = (df
    .query('SETOR_ATIV != @removed_sectors')
    .reset_index(drop=True)
)
print('Number of companies available for backtesting', df_included.CD_CVM.nunique())
df_included

Number of companies available for backtesting 1443


,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
0,ACOPALMA CIA INDL ACOS V P...,1978-05-09,2007-08-07,CANCELADA,2007-08-07,60,Metalurgia e Siderurgia,PRIVADO
1,ACOS ANHANGUERA SA,1971-10-20,1994-02-09,CANCELADA,1994-02-09,78,Metalurgia e Siderurgia,PRIVADO
2,ACOS DO BRASIL SA IND E COM,1972-08-16,1989-12-14,CANCELADA,1989-12-14,86,Metalurgia e Siderurgia,PRIVADO
...,...,...,...,...,...,...,...,...
1440,SELF IT ACADEMIAS HOLDING ...,2022-05-13,NaT,ATIVO,2022-05-13,26794,Emp. Adm. Part. - Brinqued...,PRIVADO
1441,QESTRA TECNOLOGIA ADMINIST...,2022-05-30,NaT,ATIVO,2022-05-30,26816,Comunicação e Informática,PRIVADO
1442,CONCESSIONARIA CATARINENSE...,2022-06-21,NaT,ATIVO,2022-06-21,26859,Emp. Adm. Part. - Serviços...,PRIVADO


In [13]:
df_excluded.query('DENOM_SOCIAL.str.contains("SANTANDER")')

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
48,BANCO SANTANDER NOROESTE SA,1977-07-20,1999-06-30,CANCELADA,1999-06-30,1368,Bancos,PRIVADO
68,SANTANDER LEASING S.A. ARR...,1981-11-10,2002-12-06,CANCELADA,2002-12-06,1830,Arrendamento Mercantil,PRIVADO
253,SANTANDER BRASIL ARRENDAME...,1995-07-06,2009-11-30,CANCELADA,2009-11-30,14940,Arrendamento Mercantil,PRIVADO


In [14]:
df_included.query('DENOM_SOCIAL.str.contains("SANTANDER")')

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO


In [16]:
df_excluded.to_csv("../data/excluded_companies.psv", index=False, sep="|")
df_included.to_csv("../data/included_companies.psv", index=False, sep="|")

In [17]:
# Test file
pd.read_csv("../data/included_companies.psv", sep="|")

,DENOM_SOCIAL,DT_REG,DT_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,CONTROLE_ACIONARIO
0,ACOPALMA CIA INDL ACOS V P...,1978-05-09,2007-08-07,CANCELADA,2007-08-07,60,Metalurgia e Siderurgia,PRIVADO
1,ACOS ANHANGUERA SA,1971-10-20,1994-02-09,CANCELADA,1994-02-09,78,Metalurgia e Siderurgia,PRIVADO
2,ACOS DO BRASIL SA IND E COM,1972-08-16,1989-12-14,CANCELADA,1989-12-14,86,Metalurgia e Siderurgia,PRIVADO
...,...,...,...,...,...,...,...,...
1440,SELF IT ACADEMIAS HOLDING ...,2022-05-13,NaN,ATIVO,2022-05-13,26794,Emp. Adm. Part. - Brinqued...,PRIVADO
1441,QESTRA TECNOLOGIA ADMINIST...,2022-05-30,NaN,ATIVO,2022-05-30,26816,Comunicação e Informática,PRIVADO
1442,CONCESSIONARIA CATARINENSE...,2022-06-21,NaN,ATIVO,2022-06-21,26859,Emp. Adm. Part. - Serviços...,PRIVADO
